In [1]:
import os, glob, json
import networkx as nx

In [2]:
# Get the JSON files
json_paths = glob.glob("../bddl/generated_data/transition_map/tm_jsons/*.json")
data = []
for jp in json_paths:
    with open(jp) as f:
        data.append(json.load(f))
transitions = [rule for rules in data for rule in rules]

In [3]:
transitions

[{'rule_name': 'uncooked-half__agave.n.01-dicing',
  'input_objects': {'half__agave.n.01': 1},
  'input_states': {'half__agave.n.01': [['cooked', False]]},
  'output_objects': {'diced__agave.n.01': 1}},
 {'rule_name': 'uncooked-half__antipasto.n.01-dicing',
  'input_objects': {'half__antipasto.n.01': 1},
  'input_states': {'half__antipasto.n.01': [['cooked', False]]},
  'output_objects': {'diced__antipasto.n.01': 1}},
 {'rule_name': 'cooked-half__antipasto.n.01-dicing',
  'input_objects': {'half__antipasto.n.01': 1},
  'input_states': {'half__antipasto.n.01': [['cooked', True]]},
  'output_objects': {'cooked__diced__antipasto.n.01': 1}},
 {'rule_name': 'uncooked-half__apple.n.01-dicing',
  'input_objects': {'half__apple.n.01': 1},
  'input_states': {'half__apple.n.01': [['cooked', False]]},
  'output_objects': {'diced__apple.n.01': 1}},
 {'rule_name': 'cooked-half__apple.n.01-dicing',
  'input_objects': {'half__apple.n.01': 1},
  'input_states': {'half__apple.n.01': [['cooked', True]]}

In [6]:
import collections
c = collections.Counter(x["rule_name"] for x in transitions)
[t for t in transitions if c[t["rule_name"]] > 1]

[{'rule_name': 'fruit punch',
  'input_objects': {'orange_juice.n.01': 1,
   'ginger_beer.n.01': 1,
   'lemonade.n.01': 1,
   'pineapple_juice.n.01': 1},
  'input_states': None,
  'output_objects': {'fruit_punch.n.01': 1},
  'output_states': None},
 {'rule_name': 'drip coffee',
  'input_objects': {'instant_coffee.n.01': 1, 'water.n.06': 1},
  'input_states': None,
  'output_objects': {'drip_coffee.n.01': 1},
  'output_states': None},
 {'rule_name': 'drip coffee',
  'input_objects': {'coffee_bean.n.01': 1, 'water.n.06': 1},
  'input_states': None,
  'machine': {'coffee_maker.n.01': 1},
  'output_objects': {'drip_coffee.n.01': 1},
  'output_states': {'coffee_maker.n.01,drip_coffee.n.01': [['contains',
     True]]}},
 {'rule_name': 'smoothie',
  'input_objects': {'strawberry.n.01': 3,
   'ice_cube.n.01': 4,
   'lemon_juice.n.01': 1},
  'input_states': None,
  'machine': {'blender.n.01': 1},
  'output_objects': {'smoothie.n.02': 1},
  'output_states': None},
 {'rule_name': 'fruit punch',
 

In [26]:
# Build the transition network
G = nx.DiGraph()
for transition in transitions:
    rule_name = transition["rule_name"]
    G.add_node(rule_name, type="rule")
    for input_obj in transition["input_objects"].keys():
        G.add_node(input_obj, type="obj")
        G.add_edge(input_obj, rule_name)
    for output_obj in transition["output_objects"].keys():
        G.add_node(output_obj, type="obj")
        G.add_edge(rule_name, output_obj)

In [40]:
def synset_is_reachable(s, available_objs):
    # If it's already available, then we're good.
    if s in available_objs:
        print(s, "is already available in available objects set")
        return True
    
    # Otherwise, are there any recipes that I can use to obtain it?
    for recipe, _ in G.in_edges(s):
        print("Considering recipe", recipe, "to obtain", s)
        if all(synset_is_reachable(ingredient, available_objs) for ingredient, _ in G.in_edges(recipe)):
            print("Recipe", recipe, "was usable to obtain", s)
            return True
        
    return False

def is_reachable(initial_objs, goal_objs):
    return all(synset_is_reachable(s, initial_objs) for s in goal_objs)

In [41]:
for f, t in G.subgraph(nx.dfs_tree(G.reverse(), "cooked__diced__meat_loaf.n.01").nodes).edges:
    f_name = f if G.nodes[f]['type'] == "obj" else "%s(%s)" % (f, f)
    t_name = t if G.nodes[t]['type'] == "obj" else "%s(recipe: %s)" % (t, t)
    print(f"{f_name} --> {t_name}")

brown_sugar.n.01 --> meatloaf(recipe: meatloaf)
meat_loaf.n.01 --> meat_loaf.n.01-slicing(recipe: meat_loaf.n.01-slicing)
diced__vidalia_onion.n.01 --> meatloaf(recipe: meatloaf)
whole_milk.n.01 --> meatloaf(recipe: meatloaf)
breadcrumb.n.01 --> meatloaf(recipe: meatloaf)
uncooked-half__meat_loaf.n.01-dicing(uncooked-half__meat_loaf.n.01-dicing) --> diced__meat_loaf.n.01
cooked-half__meat_loaf.n.01-dicing(cooked-half__meat_loaf.n.01-dicing) --> cooked__diced__meat_loaf.n.01
meatloaf(meatloaf) --> meat_loaf.n.01
diced__meat_loaf.n.01-cooking(diced__meat_loaf.n.01-cooking) --> cooked__diced__meat_loaf.n.01
half__vidalia_onion.n.01 --> uncooked-half__vidalia_onion.n.01-dicing(recipe: uncooked-half__vidalia_onion.n.01-dicing)
meat_loaf.n.01-slicing(meat_loaf.n.01-slicing) --> half__meat_loaf.n.01
half__meat_loaf.n.01 --> uncooked-half__meat_loaf.n.01-dicing(recipe: uncooked-half__meat_loaf.n.01-dicing)
half__meat_loaf.n.01 --> cooked-half__meat_loaf.n.01-dicing(recipe: cooked-half__meat_lo

In [47]:
synset_is_reachable("cooked__diced__meat_loaf.n.01", ["diced__vidalia_onion.n.01", "brown_sugar.n.01", "whole_milk.n.01", "breadcrumb.n.01", "ground_beef.n.01"])

Considering recipe cooked-half__meat_loaf.n.01-dicing to obtain cooked__diced__meat_loaf.n.01
Considering recipe meat_loaf.n.01-slicing to obtain half__meat_loaf.n.01
Considering recipe meatloaf to obtain meat_loaf.n.01
ground_beef.n.01 is already available in available objects set ['diced__vidalia_onion.n.01', 'brown_sugar.n.01', 'whole_milk.n.01', 'breadcrumb.n.01', 'ground_beef.n.01']
whole_milk.n.01 is already available in available objects set ['diced__vidalia_onion.n.01', 'brown_sugar.n.01', 'whole_milk.n.01', 'breadcrumb.n.01', 'ground_beef.n.01']
diced__vidalia_onion.n.01 is already available in available objects set ['diced__vidalia_onion.n.01', 'brown_sugar.n.01', 'whole_milk.n.01', 'breadcrumb.n.01', 'ground_beef.n.01']
breadcrumb.n.01 is already available in available objects set ['diced__vidalia_onion.n.01', 'brown_sugar.n.01', 'whole_milk.n.01', 'breadcrumb.n.01', 'ground_beef.n.01']
brown_sugar.n.01 is already available in available objects set ['diced__vidalia_onion.n.0

True